![](https://i.imgur.com/eBRPvWB.png)

# Practical PyTorch: Summarization with a Sequence to Sequence Network and Attention on Language Model

In this project we will be teaching a neural network to automatically summarize a text into one sentence. This model is base on the sequende-to-sequence landguage moddel of Practical Pytorch Series. All the code is tested under the Python3 environment. We use the **DUC2004 dataset** for quick test of the model.

```
[KEY: > input(description), = target(headline), < output]

> a south korean lawmaker said friday communist north korea could be producing plutonium and could have more secret underground nuclear facilities than already feared .
= s . korea says n . korea could be producing plutonium in underground plants .
< n korea korea korea n . korea korea korea producing plutonium . <EOS>

> egyptian president hosni mubarak met here sunday with syrian president hafez assad to try to defuse growing tension between syria and turkey .
= egypt s mubarak attempts mediation of turkey syria dispute
< egypt s mubarak attempts mediation of turkey syria dispute <EOS>

> police and soldiers on friday blocked off the street in front of a house where members of a terrorist gang are believed to have assembled the bomb that blew up the u .s . embassy killing people .
= suspects in us embassy bombings arrested in dar es salaam and new york
< nobel police american in operative business thai to to to to . <EOS>

> premier romano prodi battled tuesday for any votes freed up from a split in a far left party but said he will resign if he loses a confidence vote expected later this week .
= prodi faces confidence vote break with prodi divides refounding party .
< ecevit confidence confidence vote with business divides be be be . . <EOS>
```

... to varying degrees of success.

This is made possible by the simple but powerful idea of the [sequence to sequence network](http://arxiv.org/abs/1409.3215), in which two recurrent neural networks work together to transform one sequence to another. An encoder network condenses an input sequence into a single vector, and a decoder network unfolds that vector into a new sequence. 

To improve upon this model we'll use an [attention mechanism](https://arxiv.org/abs/1409.0473), which lets the decoder learn to focus over a specific range of the input sequence. This is initially designed for data-driven language translation, in which case the length of input and output sequence is approximately similar. For text summarization task, in which case the output sequence is much shorter then the input, the model didn't perform that well. Some other tricks should be explored soon. 

# The Sequence to Sequence model

A [Sequence to Sequence network](http://arxiv.org/abs/1409.3215), or seq2seq network, or [Encoder Decoder network](https://arxiv.org/pdf/1406.1078v3.pdf), is a model consisting of two separate RNNs called the **encoder** and **decoder**. The encoder reads an input sequence one item at a time, and outputs a vector at each step. The final output of the encoder is kept as the **context** vector. The decoder uses this context vector to produce a sequence of outputs one step at a time.

![](https://i.imgur.com/tVtHhNp.png)

When using a single RNN, there is a one-to-one relationship between inputs and outputs. We would quickly run into problems with different sequence orders and lengths that are common during translation. Consider the simple sentence "Je ne suis pas le chat noir" &rarr; "I am not the black cat". Many of the words have a pretty direct translation, like "chat" &rarr; "cat". However the differing grammars cause words to be in different orders, e.g. "chat noir" and "black cat". There is also the "ne ... pas" &rarr; "not" construction that makes the two sentences have different lengths.

With the seq2seq model, by encoding many inputs into one vector, and decoding from one vector into many outputs, we are freed from the constraints of sequence order and length. The encoded sequence is represented by a single vector, a single point in some N dimensional space of sequences. In an ideal case, this point can be considered the "meaning" of the sequence.

This idea can be extended beyond sequences. Image captioning tasks take an [image as input, and output a description](https://arxiv.org/abs/1411.4555) of the image (img2seq). Some image generation tasks take a [description as input and output a generated image](https://arxiv.org/abs/1511.02793) (seq2img). These models can be referred to more generally as "encoder decoder" networks.

## The Attention Mechanism

The fixed-length vector carries the burden of encoding the the entire "meaning" of the input sequence, no matter how long that may be. With all the variance in language, this is a very hard problem. Imagine two nearly identical sentences, twenty words long, with only one word different. Both the encoders and decoders must be nuanced enough to represent that change as a very slightly different point in space.

The **attention mechanism** [introduced by Bahdanau et al.](https://arxiv.org/abs/1409.0473) addresses this by giving the decoder a way to "pay attention" to parts of the input, rather than relying on a single vector. For every step the decoder can select a different part of the input sentence to consider.

![](https://i.imgur.com/5y6SCvU.png)

Attention is calculated with another feedforward layer in the decoder. This layer will use the current input and hidden state to create a new vector, which is the same size as the input sequence (in practice, a fixed maximum length). This vector is processed through softmax to create *attention weights*, which are multiplied by the encoders' outputs to create a new context vector, which is then used to predict the next output.

![](https://i.imgur.com/K1qMPxs.png)

# Requirements

You will need [PyTorch](http://pytorch.org/) to build and train the models, and [matplotlib](https://matplotlib.org/) for plotting training and visualizing attention outputs later.

In [1]:
import unicodedata
import string
import re
import random
import time
import math
import datetime
import csv

'''
import socket

hostname = socket.gethostname()
'''

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence#, masked_cross_entropy
from masked_cross_entropy import *

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline


import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


Here we will also define a constant to decide whether to use the GPU (with CUDA specifically) or the CPU. **If you don't have a GPU, set this to `False`**. Later when we create tensors, this variable will be used to decide whether we keep them on CPU or move them to GPU.

In [2]:
USE_CUDA = True

# Loading data files

The data for this project is a set of many thousands of English to French translation pairs.

[This question on Open Data Stack Exchange](http://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages) pointed me to the open translation site http://tatoeba.org/ which has downloads available at http://tatoeba.org/eng/downloads - and better yet, someone did the extra work of splitting language pairs into individual text files here: http://www.manythings.org/anki/

The English to French pairs are too big to include in the repo, so download `fra-eng.zip`, extract the text file in there, and rename it to `data/eng-fra.txt` before continuing (for some reason the zipfile is named backwards). The file is a tab separated list of translation pairs:

```
I am cold.    Je suis froid.
```

For this summarization experiment, we adopt **DUC2003 dataset**, which is stored as the `/data/desc.txt` and `/data/head.txt` for input and target file. All the lines in these two files are seprerated by `\n` and are in the same order between themselves.

Similar to the character encoding used in the character-level RNN tutorials, we will be representing each word in a language as a one-hot vector, or giant vector of zeros except for a single one (at the index of the word). Compared to the dozens of characters that might exist in a language, there are many many more words, so the encoding vector is much larger. We will however cheat a bit and trim the data to only use a few thousand words per language. On other tutorials,  they adopt an initial embedding matrix built from [GloVe](http://nlp.stanford.edu/projects/glove/), which can be tested afterwards.

###Sentence Embeddings
We'll use the gensim's library of pretrained sentence-to-vector models originally trained on the AP News dataset to use as a sentence-level analogue to semantic word embeddings. This is used as the second layer of the hierarchical attention model.

In [3]:
import gensim

sent2vec = gensim.models.doc2vec.Doc2Vec.load("../model/apnews_sen_model.model")

In [4]:
sent2vec.infer_vector("Hark the herald angels sing")

array([ 0.04134078, -0.10759926,  0.12136581,  0.11634611, -0.0058831 ,
        0.0277046 ,  0.01217325, -0.02326146, -0.37760136, -0.08064919,
        0.11972572, -0.0815114 , -0.03380077, -0.13174652, -0.31205538,
        0.01803454, -0.06165672,  0.02347835,  0.14703062,  0.06073302,
       -0.10941842, -0.00665376, -0.10876463,  0.01819875,  0.08746847,
       -0.09527182, -0.06350891, -0.08727483, -0.0623194 ,  0.08227149,
        0.02275385, -0.10708266, -0.10011625, -0.16963734, -0.08975078,
        0.13944197, -0.09122316,  0.13822769, -0.01618034, -0.35222384,
       -0.05074617, -0.12213996,  0.24208337,  0.48305428,  0.17302538,
       -0.13785513, -0.08294008,  0.25045741,  0.14873938,  0.01463736,
       -0.03235133, -0.06428432,  0.08411759,  0.13402368,  0.04845813,
        0.12466098, -0.09215906,  0.13726173, -0.03109217, -0.20206513,
       -0.1282537 ,  0.14958365, -0.05071995,  0.0846238 , -0.16750932,
       -0.06923412,  0.19461922, -0.18020755,  0.05094976, -0.08

This is just to see what the data looks like.

In [5]:
import pandas as pd
# note. this is the wrong dataset right now. (I think)
data_df = pd.read_csv('../data/shortened_out.csv')
data_df.head()

,0,The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7.,"The Daman and Diu administration on Wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media. It has been decided to celebrate the festival of Rakshabandhan on August 7. s administration was forced to retreat within 24 hours of issuing the circular that made it compulsory for its staff to celebrate Rakshabandhan at workplace. In this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues,? were issued by the Daman and Diu administration a day apart."
0,1,The administration was forced to withdraw the ...,s administration was forced to retreat within ...
1,2,Malaika Arora slammed an Instagram user who tr...,were issued by the Daman and Diu administratio...
2,3,"""Her life now is all about wearing short cloth...","In this connection, all offices/ departments s..."
3,4,"Malaika responded, ""You certainly got to get y...","the order, issued on August 1 by Gurpreet Sing..."
4,5,The Indira Gandhi Institute of Medical Science...,"the order, issued on August 1 by Gurpreet Sing..."


In [5]:
from torchtext import data
import torchtext.vocab as vocab

We first define a Field, this is a class that contains information on how you want the data preprocessed. It acts like an instruction manual that data.TabularDataset will use to parse each coloum of data input

In [6]:
TEXT = data.Field(sequential=True, init_token='<SOS>', eos_token='<EOS>',lower=True)

Read data from csv file (that is in form: csv index, summary, orginal story) into train / test splits. Each of the the three coloumns will turn into an attribute of the examples in train test objects.

In [7]:
train, test = data.TabularDataset.splits(
        path='../data/', train='out.csv',
         test='shortened_out.csv', format='csv', 
        fields=[('useless_number', TEXT),('sum', TEXT), ('story', TEXT)])

Since the fields turn into atrributes of example objects in train test split you should be able to access the summary of test example zero via: 

In [8]:
list_of_example_words = train.examples[0].sum
# make it print pretty
' '.join(list_of_example_words)

'the administration of union territory daman and diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of rakshabandhan on august 7.'

Construct the Vocab object for the TEXT field from the test dataset. (should be from train obvi but not only is this the wrong dataset something is wrong with the out.csv file that is the input to the train split)

In [9]:
TEXT.build_vocab(test, vectors="glove.6B.100d")

Each of the vectors below is the word vector embedding for each of the words in our vocabulary. to see the full vocabulary uncomment the line below vocab.vectors

In [10]:
vocab = TEXT.vocab
vocab.vectors.shape
#vocab.stoi

torch.Size([1233, 100])

get a vector pair for an example

There are two parts to understand to operate on a batch:

* Create a batch iterator
* For each batch, embed the numerizied words into word vecs using the glove vectors from the vocab 

In [12]:
def get_iterator_over_batches(dataset, batch_size, train=True, shuffle=True, repeat=False):
    dataset_iter = data.BucketIterator(
        dataset, batch_size=batch_size, device=-1, 
        sort_key=lambda x: len(x.story),
        train=train, shuffle=shuffle, repeat=repeat,
        sort=False
    )
    dataset_iter.create_batches()
    return dataset_iter

#make bucket iterator 
b_iter = get_iterator_over_batches(test, 32)

for batch in b_iter:
    print('size of batch' , batch.batch_size, '|| len of examples in batch', len(batch.story))
    
    # examples come out of the batch numericalized in a batch of (num_words_in_padded_sen X batch_size)
    print("Numericalized Batch")
    print(batch.story)
    
    # We can decode an example back into a list of words to see that these values are reasonable
    example = batch.story[:,0].data.tolist()
    print('decoded example from batch:')
    print([vocab.itos[e] for e in example])
    
    # We can create a example embedding layer to see what happens when we use the glove vectors properly
    embed = nn.Embedding(len(vocab), 100)
    embed.weight.data.copy_(vocab.vectors)
    
    print("Shape of embedded batch")
    embedding = embed(batch.story) # Embedded batch will be (num_words X batch_size X glove_dim)   
    print(embedding.shape)
    break

size of batch 32 || len of examples in batch 103
Numericalized Batch
Variable containing:
    2     2     2  ...      2     2     2
    4    62    28  ...     29    29     4
  114    22    36  ...     60    60    56
       ...          ⋱          ...       
    1     1     1  ...     21    54    25
    1     1     1  ...     85    52    99
    1     1     1  ...      3     3     3
[torch.LongTensor of size 103x32]

decoded example from batch:
['<SOS>', 'the', 'two', 'notifications', '?', 'it', 'has', 'been', 'decided', 'to', 'celebrate', 'the', 'festival', 'of', 'rakshabandhan', 'on', 'august', '7.', 'one', 'mandating', 'the', 'celebration', 'of', 'rakshabandhan', '(left)', 'and', 'the', 'other', 'withdrawing', 'the', 'mandate', '(right)', '?', 'the', 'union', 'territory?', 's', 'administration', 'was', 'forced', 'to', 'retreat', 'within', '24', 'hours', 'of', 'issuing', 'the', 'circular', 'that', 'made', 'it', 'compulsory', 'for', 'its', 'staff', 'to', 'celebrate', 'rakshabandhan', 'a

# Building the models

## The Encoder

<img src="images/encoder-network.png" style="float: right" />

The encoder will take a batch of word sequences, a `LongTensor` of size `(max_len x batch_size)`, and output an encoding for each word, a `FloatTensor` of size `(max_len x batch_size x hidden_size)`.

The word inputs are fed through an [embedding layer `nn.Embedding`](http://pytorch.org/docs/nn.html#embedding) to create an embedding for each word, with size `seq_len x hidden_size` (as if it was a batch of words). This is resized to `seq_len x 1 x hidden_size` to fit the expected input of the [GRU layer `nn.GRU`](http://pytorch.org/docs/nn.html#gru). The GRU will return both an output sequence of size `seq_len x hidden_size`.

In [13]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.embedding.weight.data.copy_(vocab.vectors)

        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=self.dropout, bidirectional=True)
        
    def forward(self, input_seqs, input_lengths, hidden=None):
        # Note: we run this all at once (over multiple batches of multiple sequences)
        embedded = self.embedding(input_seqs)
        # print(embedded.size())
        #packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(embedded, hidden)
        # outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
        return outputs, hidden

## Attention Decoder

### Interpreting the Bahdanau et al. model

The attention model in [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473) is described as the following series of equations.

Each decoder output is conditioned on the previous outputs and some $\mathbf x$, where $\mathbf x$ consists of the current hidden state (which takes into account previous outputs) and the attention "context", which is calculated below. The function $g$ is a fully-connected layer with a nonlinear activation, which takes as input the values $y_{i-1}$, $s_i$, and $c_i$ concatenated.

$$
p(y_i \mid \{y_1,...,y_{i-1}\},\mathbf{x}) = g(y_{i-1}, s_i, c_i)
$$

The current hidden state $s_i$ is calculated by an RNN $f$ with the last hidden state $s_{i-1}$, last decoder output value $y_{i-1}$, and context vector $c_i$.

In the code, the RNN will be a `nn.GRU` layer, the hidden state $s_i$ will be called `hidden`, the output $y_i$ called `output`, and context $c_i$ called `context`.

$$
s_i = f(s_{i-1}, y_{i-1}, c_i)
$$

The context vector $c_i$ is a weighted sum of all encoder outputs, where each weight $a_{ij}$ is the amount of "attention" paid to the corresponding encoder output $h_j$.

$$
c_i = \sum_{j=1}^{T_x} a_{ij} h_j
$$

... where each weight $a_{ij}$ is a normalized (over all steps) attention "energy" $e_{ij}$ ...

$$
a_{ij} = \dfrac{exp(e_{ij})}{\sum_{k=1}^{T} exp(e_{ik})}
$$

... where each attention energy is calculated with some function $a$ (such as another linear layer) using the last hidden state $s_{i-1}$ and that particular encoder output $h_j$:

$$
e_{ij} = a(s_{i-1}, h_j)
$$

### Implementing an attention module

[Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025) by Luong et al. describe a few more attention models that offer improvements and simplifications. They describe a few "global attention" models, the distinction between them being the way the attention scores are calculated.

The general form of the attention calculation relies on the target (decoder) side hidden state and corresponding source (encoder) side state, normalized over all states to get values summing to 1:

$$
a_t(s) = align(h_t, \bar h_s)  = \dfrac{exp(score(h_t, \bar h_s))}{\sum_{s'} exp(score(h_t, \bar h_{s'}))}
$$

The specific "score" function that compares two states is either *dot*, a simple dot product between the states; *general*, a a dot product between the decoder hidden state and a linear transform of the encoder state; or *concat*, a dot product between a new parameter $v_a$ and a linear transform of the states concatenated together.

$$
score(h_t, \bar h_s) =
\begin{cases}
h_t ^\top \bar h_s & dot \\
h_t ^\top \textbf{W}_a \bar h_s & general \\
v_a ^\top \textbf{W}_a [ h_t ; \bar h_s ] & concat
\end{cases}
$$

The modular definition of these scoring functions gives us an opportunity to build specific attention module that can switch between the different score methods. The input to this module is always the hidden state (of the decoder RNN) and set of encoder outputs.

In [14]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        
        self.method = method
        self.hidden_size = hidden_size
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)

        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(1, hidden_size))

    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(0)
        this_batch_size = encoder_outputs.size(1)

        # Create variable to store attention energies
        attn_energies = Variable(torch.zeros(this_batch_size, max_len)) # B x S
        # print('hidden: ', hidden)
        # print('encoder_outputs: ', encoder_outputs)
        
        if USE_CUDA:
            attn_energies = attn_energies.cuda()

        # For each batch of encoder outputs
        for b in range(this_batch_size):
            # Calculate energy for each encoder output
            for i in range(max_len):
                attn_energies[b, i] = self.score(hidden[b].unsqueeze(0), encoder_outputs[i, b].unsqueeze(0))

        # Normalize energies to weights in range 0 to 1, resize to 1 x B x S
        return F.softmax(attn_energies).unsqueeze(1)
    
    def score(self, hidden, encoder_output):
        
        if self.method == 'dot':
            energy = torch.dot(hidden.view(-1), energy.view(-1))
            return energy
        
        elif self.method == 'general':
            energy = self.attn(encoder_output)
            energy = torch.dot(hidden.view(-1), energy.view(-1))
            
            return energy
        
        elif self.method == 'concat':
            #print('hidden: ', hidden)
            #print('encoder_output: ', encoder_output)
            energy = self.attn(torch.cat((hidden, encoder_output), 1))
            energy = torch.dot(self.v.contiguous().view(-1), energy.contiguous().view(-1))
            return energy

### Implementing the Bahdanau et al. model

In summary our decoder should consist of four main parts - an embedding layer turning an input word into a vector; a layer to calculate the attention energy per encoder output; a RNN layer; and an output layer.

The decoder's inputs are the last RNN hidden state $s_{i-1}$, last output $y_{i-1}$, and all encoder outputs $h_*$.

* embedding layer with inputs $y_{i-1}$
    * `embedded = embedding(last_rnn_output)`
* attention layer $a$ with inputs $(s_{i-1}, h_j)$ and outputs $e_{ij}$, normalized to create $a_{ij}$
    * `attn_energies[j] = attn_layer(last_hidden, encoder_outputs[j])`
    * `attn_weights = normalize(attn_energies)`
* context vector $c_i$ as an attention-weighted average of encoder outputs
    * `context = sum(attn_weights * encoder_outputs)`
* RNN layer(s) $f$ with inputs $(s_{i-1}, y_{i-1}, c_i)$ and internal hidden state, outputting $s_i$
    * `rnn_input = concat(embedded, context)`
    * `rnn_output, rnn_hidden = rnn(rnn_input, last_hidden)`
* an output layer $g$ with inputs $(y_{i-1}, s_i, c_i)$, outputting $y_i$
    * `output = out(embedded, rnn_output, context)`

In [15]:
class BahdanauAttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1):
        super(BahdanauAttnDecoderRNN, self).__init__()
        
        # Define parameters
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        # Define layers

        self.embedding = nn.Embedding(output_size, hidden_size)
        # output_size of decoder should be in vocab dimensonality
        self.embedding.weight.data.copy_(vocab.vectors)

        self.dropout = nn.Dropout(dropout_p)
        self.attn = Attn('concat', hidden_size)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, n_layers, dropout=dropout_p)
        self.out = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, input_seq, last_hidden, encoder_outputs):
        # Note: we run this one step at a time
        # TODO: FIX BATCHING
        batch_size = input_seq.size(0)
        # print(input_seq)
        # Get the embedding of the current input word (last output word)
        embedded = self.embedding(input_seq)
        embedded = self.dropout(embedded)
        embedded = embedded.view(1, batch_size, self.hidden_size) # S=1
        
#         embedded = self.embedding(input_seq).view(1, 1, -1) # S=1 x B x N
#         embedded = self.dropout(embedded)
   
        # Calculate attention weights and apply to encoder outputs
        # print(encoder_outputs.size())
        # print(last_hidden.size())

        attn_weights = self.attn(last_hidden[-1], encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x 1 x N
        context = context.transpose(0, 1) # 1 x B x N
        
        # Combine embedded input word and attended context, run through RNN
        rnn_input = torch.cat((embedded, context), 2)
        output, hidden = self.gru(rnn_input, last_hidden)
        
        # Final output layer
        output = output.squeeze(0) # B x N
        context = context.squeeze(0) # B x N

#         print("Context size: ", context.size())
#         print("RNN out size: ", output.size())
#         print("Output cat Context size: ", torch.cat((output, context), 1).shape)
#         print("Output size: ", self.out(torch.cat((output, context), 1)).shape)
        output = F.log_softmax(self.out(torch.cat((output, context), 1)))
        
        # Return final output, hidden state, and attention weights (for visualization)
        return output, hidden, attn_weights

Now we can build a decoder that plugs this Attn module in after the RNN to calculate attention weights, and apply those weights to the encoder outputs to get a context vector.

### Testing the models

To make sure the Encoder and Decoder model are working (and working together) we'll do a quick test with fake word inputs:

In [16]:
small_batch_size = 32
b_iter = get_iterator_over_batches(test, small_batch_size)
print('have iter')

for batch in b_iter:
    input_batches = batch.story
    input_lengths = len(batch.story)
    target_batches = batch.sum
    target_lengths = len(batch.sum)
    print()
    if USE_CUDA:
        input_batches = input_batches.cuda()
        target_batches = target_batches.cuda()
    # input_batches, input_lengths, target_batches, target_lengths = random_batch(small_batch_size)

    print('input_batches', input_batches.size()) # (max_len x batch_size)
    print('target_batches', target_batches.size()) # (max_len x batch_size)

have iter

input_batches torch.Size([103, 32])
target_batches torch.Size([35, 32])

input_batches torch.Size([125, 4])
target_batches torch.Size([29, 4])

input_batches torch.Size([125, 32])
target_batches torch.Size([41, 32])

input_batches torch.Size([111, 32])
target_batches torch.Size([37, 32])


/home/kabirm2/anaconda3/envs/pytorchcuda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  """


Create models with a small size (a good idea for eyeball inspection):

In [17]:
small_hidden_size = 100
small_n_layers = 2

encoder_test = EncoderRNN(len(vocab), small_hidden_size, small_n_layers)
decoder_test = BahdanauAttnDecoderRNN(small_hidden_size, len(vocab), small_n_layers)

if USE_CUDA:
    encoder_test.cuda()
    decoder_test.cuda()
    
print(encoder_test)
print(decoder_test)

EncoderRNN(
  (embedding): Embedding(1233, 100)
  (gru): GRU(100, 100, num_layers=2, dropout=0.1, bidirectional=True)
)
BahdanauAttnDecoderRNN(
  (embedding): Embedding(1233, 100)
  (dropout): Dropout(p=0.1)
  (attn): Attn(
    (attn): Linear(in_features=200, out_features=100)
  )
  (gru): GRU(200, 100, num_layers=2, dropout=0.1)
  (out): Linear(in_features=200, out_features=1233)
)


To test the encoder, run the input batch through to get per-batch encoder outputs:

In [18]:
input_lengths

111

In [19]:
encoder_outputs, encoder_hidden = encoder_test(input_batches, input_lengths, None)

print('encoder_outputs', encoder_outputs.size()) # max_len x batch_size x hidden_size
print('encoder_hidden', encoder_hidden.size()) # n_layers * 2 x batch_size x hidden_size

encoder_outputs torch.Size([111, 32, 100])
encoder_hidden torch.Size([4, 32, 100])


Then starting with a SOS token, run word tokens through the decoder to get each next word token. Instead of doing this with the whole sequence, it is done one at a time, to support using it's own predictions to make the next prediction. This will be one time step at a time, but batched per time step. In order to get this to work for short padded sequences, the batch size is going to get smaller each time.

In [20]:
vocab.stoi['<SOS>']

2

In [21]:
max_target_length = target_lengths

# Prepare decoder input and outputs
decoder_input = Variable(torch.LongTensor([vocab.stoi['<SOS>']] * small_batch_size))
print(decoder_input.size())
decoder_hidden = encoder_hidden[:decoder_test.n_layers] # Use last (forward) hidden state from encoder
all_decoder_outputs = Variable(torch.zeros(max_target_length, small_batch_size, decoder_test.output_size))

if USE_CUDA:
    all_decoder_outputs = all_decoder_outputs.cuda()
    decoder_input = decoder_input.cuda()

# Run through decoder one time step at a time
for t in range(target_lengths):
    decoder_output, decoder_hidden, decoder_attn = decoder_test(
        decoder_input.contiguous(), decoder_hidden.contiguous(), encoder_outputs.contiguous()
    )
    all_decoder_outputs[t] = decoder_output # Store this step's outputs
    decoder_input = target_batches[t] # Next input is current target

    
print(all_decoder_outputs.size())
print(target_batches.size())

# Test masked cross entropy loss
loss = masked_cross_entropy(
    all_decoder_outputs.transpose(0, 1).contiguous(),
    target_batches.transpose(0, 1).contiguous(),
    torch.LongTensor([vocab.stoi['<SOS>']] * target_batches.size(1)),
    use_cuda=USE_CUDA
)
print('loss', loss.data[0])

torch.Size([32])


/home/kabirm2/anaconda3/envs/pytorchcuda/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/kabirm2/anaconda3/envs/pytorchcuda/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([37, 32, 1233])
torch.Size([37, 32])
loss 7.191926956176758


/home/kabirm2/abstractiveTextSummarization/seq2seq_first_bite/masked_cross_entropy.py:43: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_probs_flat = functional.log_softmax(logits_flat)


# Training

## Defining a training iteration

To train we first run the input sentence through the encoder word by word, and keep track of every output and the latest hidden state. Next the decoder is given the last hidden state of the decoder as its first hidden state, and the `<SOS>` token as its first input. From there we iterate to predict a next token from the decoder.

### Teacher Forcing and Scheduled Sampling

"Teacher Forcing", or maximum likelihood sampling, means using the real target outputs as each next input when training. The alternative is using the decoder's own guess as the next input. Using teacher forcing may cause the network to converge faster, but [when the trained network is exploited, it may exhibit instability](http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf).

You can observe outputs of teacher-forced networks that read with coherent grammar but wander far from the correct translation - you could think of it as having learned how to listen to the teacher's instructions, without learning how to venture out on its own.

The solution to the teacher-forcing "problem" is known as [Scheduled Sampling](https://arxiv.org/abs/1506.03099), which simply alternates between using the target values and predicted values when training. We will randomly choose to use teacher forcing with an if statement while training - sometimes we'll feed use real target as the input (ignoring the decoder's output), sometimes we'll use the decoder's output.

In [ ]:
def train(input_batches, input_lengths, target_batches, target_lengths, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    
    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 # Added onto for each word
    print('hello')
    # Run words through encoder
    encoder_outputs, encoder_hidden = encoder(input_batches, input_lengths, None)
    print('da fuk man')
    # Prepare input and output variables
    decoder_input = Variable(torch.LongTensor([SOS_token] * batch_size))
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder
    print('like why')
    max_target_length = max(target_lengths)
    all_decoder_outputs = Variable(torch.zeros(max_target_length, batch_size, decoder.output_size))
    print('max_target_length = ' + str(max_target_length))
    
    # Move new Variables to CUDA
    if USE_CUDA:
        decoder_input = decoder_input.cuda()
        all_decoder_outputs = all_decoder_outputs.cuda()

    # Run through decoder one time step at a time
    for t in range(max_target_length):
        decoder_output, decoder_hidden, decoder_attn = decoder(
            decoder_input, decoder_hidden, encoder_outputs
        )

        all_decoder_outputs[t] = decoder_output
        decoder_input = target_batches[t] # Next input is current target
    print('well, it made it through the decoder')
    # Loss calculation and backpropagation
    loss = masked_cross_entropy(
        all_decoder_outputs.transpose(0, 1).contiguous(), # -> batch x seq
        target_batches.transpose(0, 1).contiguous(), # -> batch x seq
        target_lengths
    )
    print('infinite backprop?')
    loss.backward()
    
    # Clip gradient norms
    ec = torch.nn.utils.clip_grad_norm(encoder.parameters(), clip)
    dc = torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)

    # Update parameters with optimizers
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0], ec, dc

Finally helper functions to print time elapsed and estimated time remaining, given the current time and progress.

In [ ]:
def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

## Running training

With everything in place we can actually initialize a network and start training.

To start, we initialize models, optimizers, and a loss function (criterion).

In [ ]:
attn_model = 'general'
hidden_size = 500
n_layers = 2
dropout = 0.1
batch_size = 100
#batch_size = 50
dropout= .1
# Initialize models
encoder = EncoderRNN(voc.n_words, hidden_size, n_layers, dropout=dropout)
decoder = BahdanauAttnDecoderRNN(hidden_size, voc.n_words, n_layers)

# Move models to GPU
if USE_CUDA:
    encoder.cuda()
    decoder.cuda()

# if torch.cuda.device_count() > 1:
#     encoder = nn.DataParallel(encoder)
#     decoder = nn.DataParallel(decoder)





Setting parameters for training

In [ ]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 0.5
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_epochs = 50000
epoch = 0
plot_every = 20
print_every = 2
evaluate_every = 100


# Initialize optimizers and criterion
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
criterion = nn.CrossEntropyLoss()

print out the progress while training

In [ ]:

# Keep track of time elapsed and running averages
start = time.time()
plot_losses = []
print_loss_total = 2 # Reset every print_every
plot_loss_total = 2 # Reset every plot_every

# Evaluating the network

Evaluation is mostly the same as training, but there are no targets. Instead we always feed the decoder's predictions back to itself. Every time it predicts a word, we add it to the output string. If it predicts the EOS token we stop there. We also store the decoder's attention outputs for each step to display later.

In [ ]:
def evaluate(input_seq):
    input_lengths = [len(input_seq)]
    input_seqs = [indexes_from_sentence(input_lang, input_seq)]
    input_batches = Variable(torch.LongTensor(input_seqs), volatile=True).transpose(0, 1)
    
    if USE_CUDA:
        input_batches = input_batches.cuda()
        
    # Set to not-training mode to disable dropout
    encoder.train(False)
    decoder.train(False)
    
    # Run through encoder
    encoder_outputs, encoder_hidden = encoder(input_batches, input_lengths, None)

    # Create starting vectors for decoder
    decoder_input = Variable(torch.LongTensor([SOS_token]), volatile=True) # SOS
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder
    
    if USE_CUDA:
        decoder_input = decoder_input.cuda()

    # Store output words and attention states
    decoded_words = []
    decoder_attentions = torch.zeros(max_length + 1, max_length + 1)
    
    # Run through decoder
    for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs
        )
        decoder_attentions[di,:decoder_attention.size(2)] += decoder_attention.squeeze(0).squeeze(0).cpu().data

        # Choose top word from output
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])
            
        # Next input is chosen word
        decoder_input = Variable(torch.LongTensor([ni]))
        if USE_CUDA: decoder_input = decoder_input.cuda()

    # Set back to training mode
    encoder.train(True)
    decoder.train(True)
    
    return decoded_words, decoder_attentions[:di+1, :len(encoder_outputs)]

We can evaluate random sentences from the training set and print out the input, target, and output to make some subjective quality judgements:

In [ ]:
def evaluate_randomly():
    [input_sentence, target_sentence] = random.choice(pairs)
    evaluate_and_show_attention(input_sentence, target_sentence)

# Visualizing attention

A useful property of the attention mechanism is its highly interpretable outputs. Because it is used to weight specific encoder outputs of the input sequence, we can imagine looking where the network is focused most at each time step.

You could simply run `plt.matshow(attentions)` to see attention output displayed as a matrix, with the columns being input steps and rows being output steps:

In [ ]:
# import io
# import torchvision
# from PIL import Image
# import visdom
# vis = visdom.Visdom()

# def show_plot_visdom():
#     buf = io.BytesIO()
#     plt.savefig(buf)
#     buf.seek(0)
#     attn_win = 'attention (%s)' % hostname
#     vis.image(torchvision.transforms.ToTensor()(Image.open(buf)), win=attn_win, opts={'title': attn_win})

For a better viewing experience we will do the extra work of adding axes and labels:

In [ ]:
def show_attention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    #show_plot_visdom()
    plt.show()
    plt.close()

In [ ]:

# def evaluate_and_show_attention(input_sentence, target_sentence=None):
#     output_words, attentions = evaluate(input_sentence)
#     output_sentence = ' '.join(output_words)
#     print('>', input_sentence)
#     if target_sentence is not None:
#         print('=', target_sentence)
#     print('<', output_sentence)
    
#     show_attention(input_sentence, output_words, attentions)
    
#     # Show input, target, output text in visdom
#     win = 'evaluted (%s)' % hostname
#     text = '<p>&gt; %s</p><p>= %s</p><p>&lt; %s</p>' % (input_sentence, target_sentence, output_sentence)
#     vis.text(text, win=win, opts={'title': win})

# Putting it all together

**TODO** Run `train_epochs` for `n_epochs`

To actually train, we call the train function many times, printing a summary as we go.

*Note:* If you're running this notebook you can **train, interrupt, evaluate, and come back to continue training**. Simply run the notebook starting from the following cell (running from the previous cell will reset the models).

In [ ]:
# Begin!
ecs = []
dcs = []
eca = 0
dca = 0

while epoch < n_epochs:
    epoch += 1
    
    # Get training data for this cycle
    input_batches, input_lengths, target_batches, target_lengths = random_batch(batch_size)

    # Run the train function
    loss, ec, dc = train(
        input_batches, input_lengths, target_batches, target_lengths,
        encoder, decoder,
        encoder_optimizer, decoder_optimizer, criterion
    )

    # Keep track of loss
    print_loss_total += loss
    plot_loss_total += loss
    eca += ec
    dca += dc
    
    print_loss_avg = print_loss_total / print_every
    print_loss_total = 0
    print_summary = '%s (%d %d%%) %.4f' % (time_since(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg)
    print(print_summary)

    ##job.record(epoch, loss)

#     if epoch % 1 == 0:
#         print_loss_avg = print_loss_total / print_every
#         print_loss_total = 0
#         print_summary = '%s (%d %d%%) %.4f' % (time_since(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg)
#         print(print_summary)
        
    if epoch % evaluate_every == 0:
        evaluate_randomly()

    if epoch % plot_every == 0:
        plot_loss_avg = plot_loss_total / plot_every
        plot_losses.append(plot_loss_avg)
        plot_loss_total = 0
        
        # TODO: Running average helper
        ecs.append(eca / plot_every)
        dcs.append(dca / plot_every)
        
        '''
        ecs_win = 'encoder grad (%s)' % hostname
        dcs_win = 'decoder grad (%s)' % hostname
        vis.line(np.array(ecs), win=ecs_win, opts={'title': ecs_win})
        vis.line(np.array(dcs), win=dcs_win, opts={'title': dcs_win})
        '''

        eca = 0
        dca = 0

## Plotting training loss

Plotting is done with matplotlib, using the array `plot_losses` that was created while training.

In [ ]:
def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2) # put ticks at regular intervals
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

show_plot(plot_losses)

In [ ]:
output_words, attentions = evaluate("premier romano prodi battled tuesday for any votes freed up from a split in a far left party but said he will resign if he loses a confidence vote expected later this week .")
plt.matshow(attentions.numpy())
show_plot_visdom()

In [ ]:
evaluate_and_show_attention("a south korean lawmaker said friday communist north korea could be producing plutonium and could have more secret underground nuclear facilities than already feared .")

In [ ]:
evaluate_and_show_attention("egyptian president hosni mubarak met here sunday with syrian president hafez assad to try to defuse growing tension between syria and turkey .")

In [ ]:
evaluate_and_show_attention("police and soldiers on friday blocked off the street in front of a house where members of a terrorist gang are believed to have assembled the bomb that blew up the u .s . embassy killing people .")

In [ ]:
evaluate_and_show_attention("premier romano prodi battled tuesday for any votes freed up from a split in a far left party but said he will resign if he loses a confidence vote expected later this week .")

# To do

* Try with a different dataset
    * cnn/dailymail
    * gigawords
    * standford
    * Human &rarr; Machine (e.g. IOT commands)
    * Chat &rarr; Response
    * Question &rarr; Answer
* Replace the embedding pre-trained word embeddings such as word2vec or GloVe
* Try with more layers, more hidden units, and more sentences. Compare the training time and results.
* Try different RNN layers like lstm.
* Add batch operation for GPU training
* Add beam search on decoder side when dealing with long documents.
* Control the Different output size
* Dig out other tricks